In [1]:
import torchvision
from torch import nn
from pathlib import Path
import time

import torch, torchvision

import os
import pathlib
import torch
import random
import matplotlib.pyplot as plt
%matplotlib inline
from typing import Tuple, List, Dict
from torch import nn

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

from timeit import default_timer as timer 

In [2]:
class_names = ['Bacterial_spot',
                 'Curl_Virus',
                 'Early_blight',
                 'Healthy',
                 'Late_blight',
                 'Leaf_Mold',
                 'Mosaic_virus',
                 'Septoria_leaf_spot',
                 'Target_Spot',
                 'Two_spotted_spider_mite']

In [4]:
%pwd
os.chdir('../')

In [5]:
loc = Path("Data\PlantVillage")

In [6]:
model = torchvision.models.efficientnet_b2()

In [9]:
saved_model = torch.load("models\model.pth", 
                         map_location=torch.device('cpu')).to('cpu')

In [10]:
data_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
import gradio as gr

c:\Users\harsh\anaconda3\envs\tomato\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from typing import Tuple, Dict

def predict(img) -> Tuple[Dict, float]:
    """Transforms and performs a prediction on img and returns prediction and time taken.
    """
    # Start the timer
    start_time = timer()
    
    # Transform the target image and add a batch dimension
    img = data_transform(img).unsqueeze(0).to('cpu')
    
    # Put model into evaluation mode and turn on inference mode
    saved_model.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(saved_model(img), dim=1)
    
    # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    # Calculate the prediction time
    pred_time = round(timer() - start_time, 5)
    
    # Return the prediction dictionary and prediction time 
    return pred_labels_and_probs, pred_time

In [16]:
import random
from PIL import Image
from pathlib import Path

# Get a list of all test image filepaths
test_data_paths = list(loc.glob("*/*.jpg"))
#image_path_list = list(loc.glob("*/*.jpg"))

# Randomly select a test image path
random_image_path = random.sample(test_data_paths, k=1)[0]

# Open the target image
image = Image.open(random_image_path)
print(f"[INFO] Predicting on image at path: {random_image_path}\n")

# Predict on the target image and print out the outputs
pred_dict, pred_time = predict(img=image)
print(f"Prediction label and probability dictionary: \n{pred_dict}")
print(f"Prediction time: {pred_time} seconds")

[INFO] Predicting on image at path: Data\PlantVillage\Two_spotted_spider_mite\70787448-85e1-4479-a11b-86c52893ff22___Com.G_SpM_FL 9368.JPG

Prediction label and probability dictionary: 
{'Bacterial_spot': 0.002127839718014002, 'Curl_Virus': 0.0002441005635773763, 'Early_blight': 0.0005701184272766113, 'Healthy': 0.00013484872761182487, 'Late_blight': 0.000562656088732183, 'Leaf_Mold': 0.0009177390020340681, 'Mosaic_virus': 0.0007484101806767285, 'Septoria_leaf_spot': 5.128438715473749e-05, 'Target_Spot': 0.017970064654946327, 'Two_spotted_spider_mite': 0.9766729474067688}
Prediction time: 0.0571 seconds


In [20]:
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

[['Data\\PlantVillage\\Target_Spot\\0e9c0af8-facc-4240-b498-41f8004c8061___Com.G_TgS_FL 0915.JPG'],
 ['Data\\PlantVillage\\Bacterial_spot\\f332d822-2557-42ee-ba97-c62cafcc5c29___GCREC_Bact.Sp 3172.JPG'],
 ['Data\\PlantVillage\\Bacterial_spot\\dd6e2b14-f790-452a-b810-a667064a8699___GCREC_Bact.Sp 5632.JPG']]

In [18]:
import gradio as gr

# Create title, description and article strings
title = "Tomato Leaf Disease "
description = "An EfficientNetB2 feature extractor computer vision model to classify images of tomato leaf if they are healthy or infected"

# Create the Gradio demo
demo = gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type="pil"), # what are the inputs?
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    examples=example_list, 
                    title=title,
                    description=description)

# Launch the demo!
demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://dd40198f8ad35e6014.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
